## IBM Data Science Capstone Project Final 
#### Business Objective : Recommend the best place to open a new Greek Restaurant in Toronto
##### By Paris Pantazis

We shall use k-mean clustering and FourSquare API to find the most suitable location to open an authentic Greek restaurant in Toronto, Canada.

In [1]:
#importing necessary libraries
import requests
import pandas as pd

In [2]:
#scrapping neighborhoods in Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [3]:
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [16]:
#Replace empty Neighborhood with Borough name and check again
df_html['Neighbourhood'].fillna(df_html['Borough'], inplace=True)
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [14]:
#Group by Postal Code / Borough
df_postcodes = df_html.groupby(['Postal Code','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
print('The shape of the dataset is:',df_postcodes.shape)

The shape of the dataset is: (103, 3)


In [18]:
#to make it easier, we will store this in csv format.
#Export to .CSV
df_postcodes.to_csv('Toronto_Postcodes.csv')

In [19]:
import numpy as np

Since I'm facing issues with GeoEncoder library, let's uise the below csv file instead

In [20]:
#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
#use the previously cleaned data
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
# Make sure both dataframes have the same 
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [28]:
# Merge both datasets
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
# Check coordinates for a couple of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [30]:
#Export to .CSV
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')

In [32]:
# Installing Folium 
!pip install folium

     |████████████████████████████████| 94 kB 4.8 MB/s  eta 0:00:01


In [33]:
import folium
from sklearn.cluster import KMeans

In [34]:
# Read .csv file from above
df = pd.read_csv('Toronto_Postcodes_2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [35]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [36]:
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [37]:
#count Bourough and Neighborhood
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [38]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [39]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [40]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

In [41]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717947, -79.38987324871795


In [42]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [43]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Getting Venues Data using Foursquare

In [45]:
CLIENT_ID = 'KDSWY0KDE4ZA114EGMOD0QFBFGUMT5Y2SWBLY11ZYFQMEHBB' # your Foursquare ID
CLIENT_SECRET = 'D5WENVTK3LGAI1DJPKX3S2IHM0S5EHVO4IXEBAT2Z1DTRXMU' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: KDSWY0KDE4ZA114EGMOD0QFBFGUMT5Y2SWBLY11ZYFQMEHBB
CLIENT_SECRET: D5WENVTK3LGAI1DJPKX3S2IHM0S5EHVO4IXEBAT2Z1DTRXMU


In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [50]:
#Check size of resulting dataframe
toronto_venues.shape

(1615, 7)

In [51]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [39]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",87,87,87,87,87,87
Christie,19,19,19,19,19,19


In [52]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [41]:
#print out the list of categories
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Asian Restaurant', 'Greek Restaurant', 'Cosmetics Shop',
       'Italian Restaurant', 'Ice Cream Shop', 'Brewery', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Pizza Place',
       'Bookstore', 'Restaurant', 'Juice Bar', 'Bubble Tea Shop', 'Diner',
       'Spa', 'Furniture / Home Store', 'Grocery Store', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Frozen Yogurt Shop',
       'American Restaurant', 'Liquor Store', 'Gym', 'Burger Joint',
       'Fish & Chips Shop', 'Park', 'Sushi Restaurant', 'Burrito Place',
       'Pet Store', 'Steakhouse', 'Fast Food Restaurant', 'Movie Theater',
       'Sandwich Place', 'Light Rail Station', 'Fish Market', 'Café',
       'Cheese Shop', 'Gay Bar', 'Seafood Restaurant',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Thai Restaurant', 'Stationery Store', 'Wine Bar',
       'Coworking Space', 'Bar', 'Latin American Restaurant',
  

In [54]:
# check if the results contain "Greek Restaurant"
"Greek Restaurant" in toronto_venues['Venue Category'].unique()

True

***Analyze Each Neighborhood***

In [55]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1615, 236)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [56]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 236)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.017241,0.00000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.066667,0.066667,0.066667,0.133333,0.066667,0.133333,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.016667,0.00000,0.000000,0.016667,0.016667
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.00,...,0.012658,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.012658,0.000000,0.025316
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.00000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.030303,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000


In [57]:
# How many GR restaurants.?
len(to_grouped[to_grouped["Greek Restaurant"] > 0])

6

Create a new dataframe to find the Greek Restaurants only

In [58]:
to_greek = to_grouped[["Neighborhoods","Greek Restaurant"]]

In [59]:
to_greek.head()

,Neighborhoods,Greek Restaurant
0,Berczy Park,0.017241
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.000000


## Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Toronto into 3 clusters.

In [60]:
# set number of clusters
toclusters = 3

to_clustering = to_greek.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

In [61]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_greek.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [62]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Greek Restaurant,Cluster Labels
0,Berczy Park,0.017241,2
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"Business reply mail Processing Centre, South C...",0.000000,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
4,Central Bay Street,0.000000,0


In [63]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1615, 9)


,Neighborhood,Greek Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,2,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.017241,2,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.017241,2,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.017241,2,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.017241,2,43.644771,-79.373306,Berczy Park,43.648048,-79.375172,Park


In [64]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1615, 9)


,Neighborhood,Greek Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17,"Kensington Market, Chinatown, Grange Park",0.000000,0,43.653206,-79.400049,Blue Banana Market,43.655669,-79.402551,Furniture / Home Store
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,The Fifth & Terrace,43.649250,-79.389320,Modern European Restaurant
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Cardio-Go,43.647017,-79.388143,Gym
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Astarté Fresh Yogurt Bar,43.647596,-79.386419,Café
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop
...,...,...,...,...,...,...,...,...,...
8,Davisville,0.030303,2,43.704324,-79.388790,Pizza Pizza,43.706138,-79.389292,Pizza Place
8,Davisville,0.030303,2,43.704324,-79.388790,Apple Tree Farmer's Market,43.700326,-79.389760,Farmers Market
8,Davisville,0.030303,2,43.704324,-79.388790,Petro-Canada,43.702269,-79.387955,Gas Station
8,Davisville,0.030303,2,43.704324,-79.388790,Shoppers Drug Mart,43.707806,-79.389893,Pharmacy


Visualize the clusters

In [66]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [67]:
# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [68]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters

In [69]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Greek Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17,"Kensington Market, Chinatown, Grange Park",0.0,0,43.653206,-79.400049,Blue Banana Market,43.655669,-79.402551,Furniture / Home Store
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,The Fifth & Terrace,43.649250,-79.389320,Modern European Restaurant
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Cardio-Go,43.647017,-79.388143,Gym
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Astarté Fresh Yogurt Bar,43.647596,-79.386419,Café
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop
...,...,...,...,...,...,...,...,...,...
13,"Garden District, Ryerson",0.0,0,43.657162,-79.378937,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
13,"Garden District, Ryerson",0.0,0,43.657162,-79.378937,JOEY Eaton Centre,43.656094,-79.381878,New American Restaurant
13,"Garden District, Ryerson",0.0,0,43.657162,-79.378937,Roots,43.653613,-79.380244,Clothing Store
13,"Garden District, Ryerson",0.0,0,43.657162,-79.378937,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,Italian Restaurant


In [70]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Greek Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Simone's Caribbean Restaurant,43.678655,-79.346582,Caribbean Restaurant
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Urban Nails,43.676668,-79.356602,Spa
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Marvel Coffee Co.,43.678630,-79.347460,Coffee Shop
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Astoria Shish Kebob House,43.677596,-79.351738,Greek Restaurant
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Momo Hut And Gardens,43.677491,-79.351516,Tibetan Restaurant
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Dough Bakeshop,43.676643,-79.356846,Bakery
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Sher-E-Punjab,43.677308,-79.353066,Indian Restaurant
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Bulk Barn,43.676790,-79.355865,Grocery Store
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,Kalyvia,43.677973,-79.351208,Greek Restaurant
36,"The Danforth West, Riverdale",0.186047,1,43.679557,-79.352188,IL FORNELLO on Danforth,43.678604,-79.346904,Italian Restaurant


In [71]:
#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Greek Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,2,43.644771,-79.373306,Crepe It Up!,43.648736,-79.371623,Creperie
0,Berczy Park,0.017241,2,43.644771,-79.373306,Alexandro's World Famous Gyros,43.641663,-79.375214,Greek Restaurant
0,Berczy Park,0.017241,2,43.644771,-79.373306,St. Lawrence Market Plaza,43.649169,-79.372330,Art Gallery
0,Berczy Park,0.017241,2,43.644771,-79.373306,Pravda Vodka Bar,43.648516,-79.374732,Cocktail Bar
0,Berczy Park,0.017241,2,43.644771,-79.373306,Olympic Cheese,43.648702,-79.371541,Cheese Shop
...,...,...,...,...,...,...,...,...,...
8,Davisville,0.030303,2,43.704324,-79.388790,Pizza Pizza,43.706138,-79.389292,Pizza Place
8,Davisville,0.030303,2,43.704324,-79.388790,Apple Tree Farmer's Market,43.700326,-79.389760,Farmers Market
8,Davisville,0.030303,2,43.704324,-79.388790,Petro-Canada,43.702269,-79.387955,Gas Station
8,Davisville,0.030303,2,43.704324,-79.388790,Shoppers Drug Mart,43.707806,-79.389893,Pharmacy


## Observations

* **Most of the greek restaurants are concentrated in cluster 1.** This represents a great opportunity and high potential areas to open new greek reasturants in clusters 0 and 2 as there is very little to no competition from existing identical business. Also, the recommendation is to avoid neighborhoods in cluster 1 which already have high concentration ofgreek reasturants and suffering from intense competition.*